In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from scipy import stats
from datetime import datetime

# Sample dataset creation
np.random.seed(0)

# Creating a DataFrame with numerical and categorical data
data = {
    'Date': pd.date_range(start='2020-01-01', periods=10, freq='M'),
    'Age': np.random.randint(18, 65, size=10),
    'Income': np.random.randint(30000, 120000, size=10),
    'AmountSpent': np.random.randint(100, 1000, size=10),
    'Category': ['A', 'B', 'A', 'B', 'C', 'A', 'C', 'B', 'A', 'C'],
    'Gender': ['Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female']
}

df = pd.DataFrame(data)
df

,Date,Age,Income,AmountSpent,Category,Gender
0,2020-01-31,62,110966,945,A,Male
1,2020-02-29,18,78600,172,B,Female
2,2020-03-31,21,69512,877,A,Male
3,2020-04-30,21,82620,215,B,Female
4,2020-05-31,57,110186,855,C,Male
5,2020-06-30,27,47089,809,A,Female
6,2020-07-31,37,62230,947,C,Male
7,2020-08-31,39,48983,531,B,Female
8,2020-09-30,54,119688,548,A,Male
9,2020-10-31,41,112457,950,C,Female


In [2]:
# Introduce missing values and outliers
df.loc[2, 'Age'] = np.nan  # Missing Age value
df.loc[4, 'Income'] = np.nan  # Missing Income value
df.loc[7, 'AmountSpent'] = 5000  # Outlier in AmountSpent
df.loc[6, 'Gender'] = np.nan  # Missing Gender value
df

,Date,Age,Income,AmountSpent,Category,Gender
0,2020-01-31,62.0,110966.0,945,A,Male
1,2020-02-29,18.0,78600.0,172,B,Female
2,2020-03-31,NaN,69512.0,877,A,Male
3,2020-04-30,21.0,82620.0,215,B,Female
4,2020-05-31,57.0,NaN,855,C,Male
5,2020-06-30,27.0,47089.0,809,A,Female
6,2020-07-31,37.0,62230.0,947,C,NaN
7,2020-08-31,39.0,48983.0,5000,B,Female
8,2020-09-30,54.0,119688.0,548,A,Male
9,2020-10-31,41.0,112457.0,950,C,Female


In [3]:
# Class to perform data cleaning
class DataCleaningPipeline:
    
    def __init__(self, df):
        self.df = df
        self.outliers_removed = 0  # Initialize outliers removed as 0
    
    # Handle missing values for numerical columns with mean and categorical columns with mode.
    def handle_missing_values(self):
        # Handle missing values for numeric columns
        numeric_cols = self.df.select_dtypes(include=[np.number]).columns
        self.df[numeric_cols] = self.df[numeric_cols].fillna(self.df[numeric_cols].mean())
        
        # Handle missing values for categorical columns
        categorical_cols = self.df.select_dtypes(include=[object, 'category']).columns
        self.df[categorical_cols] = self.df[categorical_cols].fillna(self.df[categorical_cols].mode().iloc[0])
    
    # Remove duplicate rows from the dataset.
    def remove_duplicates(self):
        self.df.drop_duplicates(inplace=True)
    
    # Handle outliers by removing rows with Z-scores > 3
    def handle_outliers(self):
        initial_shape = self.df.shape[0]  # Save the initial number of rows
        z_scores = np.abs(stats.zscore(self.df.select_dtypes(include=[np.number])))
        self.df = self.df[(z_scores < 3).all(axis=1)]  # Remove outliers
        final_shape = self.df.shape[0]  # Save the final number of rows
        self.outliers_removed = initial_shape - final_shape  # Track the number of removed outliers
        
    # Convert data types to appropriate formats.
    def convert_data_types(self):
        # Convert 'Date' to datetime
        self.df['Date'] = pd.to_datetime(self.df['Date'], errors='coerce')
        
        # Convert 'Category' to category type
        self.df['Category'] = self.df['Category'].astype('category')
    
    # Normalize numerical columns using MinMaxScaler and standardize using StandardScaler.
    def normalize_standardize_features(self):
        scaler = MinMaxScaler()
        self.df[['Age', 'Income']] = scaler.fit_transform(self.df[['Age', 'Income']])
        
        standard_scaler = StandardScaler()
        self.df[['AmountSpent']] = standard_scaler.fit_transform(self.df[['AmountSpent']])
    
    # Create new features for the dataset like 'Year', 'Month', 'AgeGroup'.
    def feature_engineering(self):
        self.df['Year'] = self.df['Date'].dt.year
        self.df['Month'] = self.df['Date'].dt.month
        self.df['AgeGroup'] = pd.cut(self.df['Age'], bins=[0, 18, 35, 50, 100], labels=['Child', 'Young', 'Middle-aged', 'Senior'])
    
    # Apply One-Hot Encoding for categorical columns and Label Encoding for binary columns.
    def handle_categorical_data(self):
        self.df = pd.get_dummies(self.df, columns=['Category'], drop_first=True)
        
        label_encoder = LabelEncoder()
        self.df['Gender'] = label_encoder.fit_transform(self.df['Gender'])
    
    # Generate a data quality report summarizing the cleaning steps.
    def generate_data_quality_report(self):
        missing_data = self.df.isnull().sum()
        duplicates = self.df.duplicated().sum()

        # Ensure everything in the report is converted to strings
        report = {
            'Missing Values': missing_data.to_string(),
            'Duplicate Rows': str(duplicates),
            'Outliers Removed': str(self.outliers_removed),  # Ensure this is converted to string
            'Data Types': self.df.dtypes.to_dict()
        }

        return report
    
    # Automate the entire data cleaning process.
    def clean_data(self):
        self.handle_missing_values()
        self.remove_duplicates()
        self.handle_outliers()
        self.convert_data_types()
        self.normalize_standardize_features()
        self.feature_engineering()
        self.handle_categorical_data()
        
        # Generate Data Quality Report
        report = self.generate_data_quality_report()
        
        return self.df, report

In [4]:
# Apply the pipeline
pipeline = DataCleaningPipeline(df)
cleaned_df, report = pipeline.clean_data()

# Display the cleaned DataFrame and the report
print(cleaned_df)
print("\nData Quality Report:")
print(report)

        Date       Age    Income  AmountSpent  Gender  Year  Month AgeGroup  \
0 2020-01-31  1.000000  0.879861    -0.141558       1  2020      1    Child   
1 2020-02-29  0.000000  0.434042    -0.727343       0  2020      2      NaN   
2 2020-03-31  0.489899  0.308861    -0.193089       1  2020      3    Child   
3 2020-04-30  0.068182  0.489414    -0.694757       0  2020      4    Child   
4 2020-05-31  0.886364  0.471913    -0.209761       1  2020      5    Child   
5 2020-06-30  0.204545  0.000000    -0.244620       0  2020      6    Child   
6 2020-07-31  0.431818  0.208557    -0.140043       0  2020      7    Child   
7 2020-08-31  0.477273  0.026089     2.931349       0  2020      8    Child   
8 2020-09-30  0.818182  1.000000    -0.442408       1  2020      9    Child   
9 2020-10-31  0.522727  0.900398    -0.137769       0  2020     10    Child   

   Category_B  Category_C  
0       False       False  
1        True       False  
2       False       False  
3        True     